# S03E08 - Gemstone Pricing

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

import optuna
from optuna.samplers import TPESampler
import seaborn as sns
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [2]:
train_df = pd.read_csv("./data/train.csv").assign(sample = 'train', original=0)

test_df = pd.read_csv("./data/test.csv").assign(sample = 'test', original=0)

original_df = pd.read_csv("./data/cubic_zirconia.csv").assign(sample = 'train', original = 1).drop(columns='Unnamed: 0')

df = pd.concat([train_df, test_df, original_df], sort=False).sample(frac=1, random_state=13)

In [3]:
def clarity_scale(df):
    df["clarity"] = df["clarity"].apply(
        lambda x: 0 if x == "IF" else 1 if x == "VVS1" else 2 if x == "VVS2" else 3 if x == "VS1" else 4 if x == "VS2" else 5 if x == "SI1" else 6 if x == "SI2" else 7).astype(np.int8)
    return df



In [4]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']])
df[['cut']] = enc.fit_transform(df[['cut']]).astype('int8')
enc = OrdinalEncoder(categories=[['J', 'I','H','G','F','E','D']])
df[['color']] = enc.fit_transform(df[['color']]).astype('int8')

In [5]:
# define ordinal encoding
#encoder = OneHotEncoder(sparse_output=False).fit(df[['color']])
#encoded = encoder.transform(df[['color']])
# transform data
#encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

#df = pd.concat([df[['id', 'sample','carat','clarity' ,'cut', 'color', 'depth', 'table', 'x', 'y', 'z','original' ,'price']]\
#                .reset_index(drop=True), encoded_df], axis=1).sample(frac=1, random_state=13)
df = clarity_scale(df)

In [6]:
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price,sample,original
174091,174091.0,0.51,4,4,4,61.7,56.0,5.14,5.17,3.18,1591.0,train,0
5599,5599.0,0.30,4,2,1,61.3,55.0,4.34,4.33,2.66,684.0,train,0
187609,187609.0,0.30,4,3,5,60.8,57.0,4.37,4.40,2.67,432.0,train,0
185622,185622.0,0.40,2,3,3,61.0,60.0,4.75,4.79,2.91,1026.0,train,0
73490,73490.0,0.53,4,4,3,62.0,55.0,5.20,5.23,3.23,1778.0,train,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25295,218868.0,0.40,4,3,0,61.5,55.0,4.74,4.78,2.93,NaN,test,0
164706,164706.0,0.73,2,6,6,63.6,56.0,5.65,5.70,3.62,2155.0,train,0
63169,256742.0,2.04,2,0,5,59.6,61.0,8.29,8.33,4.93,NaN,test,0
59883,253456.0,1.53,2,0,3,61.0,60.0,7.42,7.45,4.54,NaN,test,0


In [7]:
original_features = list(df.drop(columns=['id','price', 'sample']).columns)

In [8]:
features = original_features
#features = ['carat']
target = 'price'

train = df.query("sample == 'train'").drop(columns=['sample']).reset_index(drop=True).copy()
X_train = train[features].copy()
#X_train[cat_features] = X_train[cat_features].astype('category')
y_train = train['price']

test_df = df.query("sample == 'test'").drop(columns=['sample']).copy()
test_df = test_df[features + ['id']]
test = test_df.reset_index(drop=True)

In [11]:
mod = lgb.LGBMRegressor().fit(X_train, y_train)

In [15]:
def objective(trial):
    """Define the objective function"""

    params = {
        #'gpu_id':0,
        #'tree_method':'auto',
        'tree_method': trial.suggest_categorical('tree_method', ['exact','approx', 'hist']),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 0.1),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 0, 5),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 100, 700),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 1),
        #'enable_categorical': True,
        'eval_metric': 'rmse',
        'objective':'reg:squarederror',
        'seed':13,
    }


    cv = KFold(10, shuffle=True, random_state=13)
    fold_scores = []
    for i, (train_idx,val_idx) in enumerate(cv.split(train[features],train[target])):
        X_train, y_train = train.loc[train_idx, features],train.loc[train_idx, target]
        X_val, y_val = train.loc[val_idx, features],train.loc[val_idx, target]
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train,
                 y_train,
                 eval_set= [(X_val,y_val)],
                 verbose=0)
        
        pred_val = model.predict(X_val)
        #pred_test = model.predict(test[features])

        score = np.sqrt(mean_squared_error(y_val, pred_val))
        fold_scores.append(score)

    return np.mean(fold_scores)

In [22]:
def objective(trial):
    params = {
        #         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_int("n_estimators", 1000, 10000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000),
        "max_depth": trial.suggest_int("max_depth", 3, 14),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000),
        "max_bin": trial.suggest_int("max_bin", 100, 1000),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.001, 1),
        "min_split_gain": trial.suggest_float("min_split_gain", 0, 15),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95),
        'early_stopping_round': trial.suggest_int('early_stopping_round', 100, 700),

        "metric":"rmse",
        "categorical_features": "1,2,3,9",
        "verbose": -1
    }
    
    cv = KFold(10, shuffle=True, random_state=13)
    fold_scores = []
    for i, (train_idx,val_idx) in enumerate(cv.split(train[features],train[target])):
        X_train, y_train = train.loc[train_idx, features],train.loc[train_idx, target]
        X_val, y_val = train.loc[val_idx, features],train.loc[val_idx, target]
        
        
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train,
                 y_train,
                eval_set= [(X_val,y_val)],
                 verbose=-1)

        
        
        pred_val = model.predict(X_val)
        #pred_test = model.predict(test[features])

        score = np.sqrt(mean_squared_error(y_val, pred_val))
        fold_scores.append(score)

    return np.mean(fold_scores)


In [23]:
study = optuna.create_study(direction='minimize', sampler = TPESampler())
study.optimize(objective, n_trials=100)

[I 2023-02-25 09:55:58,239] A new study created in memory with name: no-name-b7f53182-21e9-4927-be5b-c24c960284b4
/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] min_data_in_leaf is set=9915, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9915
[LightGBM] [Warning] feature_fraction is set=0.8572586662401951, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8572586662401951
[LightGBM] [Warning] bagging_fraction is set=0.2861929218201531, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2861929218201531


/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] min_data_in_leaf is set=9915, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9915
[LightGBM] [Warning] feature_fraction is set=0.8572586662401951, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8572586662401951
[LightGBM] [Warning] bagging_fraction is set=0.2861929218201531, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2861929218201531


/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] min_data_in_leaf is set=4647, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4647
[LightGBM] [Warning] feature_fraction is set=0.7740902480391918, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7740902480391918
[LightGBM] [Warning] bagging_fraction is set=0.8448709276751496, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8448709276751496


/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] min_data_in_leaf is set=4647, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4647
[LightGBM] [Warning] feature_fraction is set=0.7740902480391918, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7740902480391918
[LightGBM] [Warning] bagging_fraction is set=0.8448709276751496, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8448709276751496


[W 2023-02-25 12:13:17,197] Trial 42 failed with parameters: {'n_estimators': 4732, 'learning_rate': 0.08296625452728301, 'num_leaves': 2820, 'max_depth': 9, 'min_data_in_leaf': 4647, 'max_bin': 242, 'reg_alpha': 0.09326612315731606, 'min_split_gain': 0.027968818581725813, 'bagging_fraction': 0.8448709276751496, 'bagging_freq': 8, 'feature_fraction': 0.7740902480391918, 'early_stopping_round': 327} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/43/mkchbv_x1qz9cxc3rr08t8c40000gp/T/ipykernel_17373/575197089.py", line 30, in objective
    model.fit(X_train,
  File "/Users/Jacob/.conda/envs/py38/lib/python3.8/site-packages/lightgbm/sklearn.py", line 895, in fit
    super().fit(X, y, sample_weight=sample_weight, init_score=init_score,
  File "/Users/Jacob/.conda/envs/py38/lib/pyth

In [10]:
def run_results(param_input):
    test_md = test.copy()

    X = train[features]
    Y = train['price'] 

    test_data = test[features]

    XGB_cv_scores, XGB_imp = list(), list()
    preds = list()

    skf = KFold(10, random_state = 13, shuffle = True)

    for train_ix, test_ix in skf.split(X, Y):

        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]

        ## Building RF model
        XGB_md = xgb.XGBRegressor(**param_input).fit(X_train, Y_train,
                                               eval_set= [(X_test,Y_test)],
                             verbose=0)
        #XGB_md.feature_importances_


        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict(X_test)
        XGB_pred_2 = XGB_md.predict(test_data)

        ## Computing roc-auc score
        XGB_cv_scores.append(np.sqrt(mean_squared_error(Y_test, XGB_pred_1)))
        preds.append(XGB_pred_2)

    #XGB_cv_score = np.mean(XGB_cv_scores)    
    print('The average RMSE over 10-folds is:', np.mean(XGB_cv_scores))
    
    return pd.DataFrame(preds)

In [8]:
params = {'tree_method': 'exact', 'max_depth': 9, 'learning_rate': 0.0028019118673335355, 'n_estimators': 4008, 'reg_lambda': 0.03396770709801297, 'gamma': 0.8546917842299727, 'subsample': 0.33198783521275654, 'colsample_bytree': 0.9804536473910398, 'min_child_weight': 5, 'early_stopping_rounds': 201, 'reg_alpha': 0.09887091760877176}

In [12]:
out_df = run_results(params)

The average RMSE over 10-folds is: 566.5574511783434


In [15]:
print(params)

{'tree_method': 'exact', 'max_depth': 9, 'learning_rate': 0.0028019118673335355, 'n_estimators': 4008, 'reg_lambda': 0.03396770709801297, 'gamma': 0.8546917842299727, 'subsample': 0.33198783521275654, 'colsample_bytree': 0.9804536473910398, 'min_child_weight': 5, 'early_stopping_rounds': 201, 'reg_alpha': 0.09887091760877176}


In [13]:
submission_file = pd.DataFrame(np.transpose(np.vstack([test_df['id'].astype('int'), pd.DataFrame(out_df).mean(axis=0)])), columns=['id', 'price'])
submission_file['id'] = submission_file['id'].astype('int')
submission_file.to_csv("./submissions/8.csv", index=False)

| Feature Set | Tuned Hyperparameters | CV (5-fold) Score | LB Score |
|---|---|---|---|
| All features + OneHotEncoded | {'max_depth': 8, 'learning_rate': 0.011372635640766581, 'n_estimators': 4999, 'reg_lambda': 0.08113388104914292, 'gamma': 0.4703233640126939, 'subsample': 0.572912477610107, 'colsample_bytree': 0.8466912196410705, 'min_child_weight': 5, 'early_stopping_rounds': 378, 'reg_alpha': 0.013284008444197512} | 0.576 | 0.584 |
| All Features - (x,y,z)| {'tree_method': 'hist', 'max_depth': 7, 'learning_rate': 0.050996117076570605, 'n_estimators': 2526, 'reg_lambda': 0.07610652509281068, 'gamma': 0.1414009438093801, 'subsample': 0.8244491165581733, 'colsample_bytree': 0.9318723417490518, 'min_child_weight': 5, 'early_stopping_rounds': 476, 'reg_alpha': 0.1425023600600033} | 0.580 | 0.589 |
| All features + OneHotEncoded w/ Original label (w/ Original Data) | {'tree_method': 'exact', 'max_depth': 8, 'learning_rate': 0.005855954960496774, 'n_estimators': 7141, 'reg_lambda': 0.037816359453527223, 'gamma': 0.07151677988518425, 'subsample': 0.7557679812273866, 'colsample_bytree': 0.5926578940053994, 'min_child_weight': 4, 'early_stopping_rounds': 208, 'reg_alpha': 0.413828386261195} | 0.568 | 0.578 |
| All features + OneHotEncoded w/ Original label (w/ Original Data) - original features | {'tree_method': 'approx', 'max_depth': 7, 'learning_rate': 0.0037171191585069555, 'n_estimators': 3663, 'reg_lambda': 0.04860253888263786, 'gamma': 0.04054562087853457, 'subsample': 0.46613454139173527, 'colsample_bytree': 0.5611846478420512, 'min_child_weight': 1, 'early_stopping_rounds': 432, 'reg_alpha': 0.08333588167604829} | 0.571 | 0.578 |
| All features + OneHotEncoded w/ Original label (w/ Original Data) + single carat column | {'tree_method': 'exact', 'max_depth': 8, 'learning_rate': 0.006208783389130772, 'n_estimators': 3513, 'reg_lambda': 0.06577619549392931, 'gamma': 0.1310414716253491, 'subsample': 0.7250297630603487, 'colsample_bytree': 0.7813594794765958, 'min_child_weight': 5, 'early_stopping_rounds': 566, 'reg_alpha': 0.18205513684157554} | 0.566 | 0.577 |
| All features (+ original) + Ordinal Encoded Carat and Cut columns | {'tree_method': 'exact', 'max_depth': 9, 'learning_rate': 0.0028019118673335355, 'n_estimators': 4008, 'reg_lambda': 0.03396770709801297, 'gamma': 0.8546917842299727, 'subsample': 0.33198783521275654, 'colsample_bytree': 0.9804536473910398, 'min_child_weight': 5, 'early_stopping_rounds': 201, 'reg_alpha': 0.09887091760877176} | 0.566 | 0.576

In [14]:
train.corr()['price']

id         -0.001392
carat       0.940673
clarity     0.179931
cut        -0.086346
depth       0.001260
table       0.167787
x           0.899137
y           0.895102
z           0.887524
original   -0.002407
price       1.000000
color_D    -0.090452
color_E    -0.113868
color_F    -0.035322
color_G     0.018188
color_H     0.083528
color_I     0.118717
color_J     0.081868
Name: price, dtype: float64

In [191]:
XGB_md = xgb.XGBRegressor(**study.best_params).fit(X_train, Y_train, verbose=0)

NameError: name 'Y_train' is not defined